In [1]:
import numpy as np
import json
import os

In [2]:
numpy_path = 'data/numpy/'
datapath = 'data/images_{0}.npy'
labelpath = 'data/labels_{0}.json'
testsize = 1000
valsize = 500

# Get all the categories

In [3]:
def file_name(file_dir): 
    names = []
    for root, dirs, files in os.walk(file_dir):
        names.append(files) 
    names = [file.split('.')[0] for file in names[0]]
    return names

In [4]:
names = file_name(numpy_path)
name_dict = {}
for i in range(len(names)):
    name_dict[i] = names[i]

In [5]:
with open('categories.json', 'w') as fd:
    json.dump(names, fd)
with open('categories_dict.json', 'w') as fd:
    json.dump(name_dict, fd)

# Combine all the numpy file together, generate category, train/val/test split

In [81]:
# generate category, data
category_train = []
category_test = []
category_val = []
image_train = None
image_test = None
image_val = None
for i in range(len(names)):
    data = np.load(numpy_path+names[i]+'.npy')
    instance_num = data.shape[0]
    
    image_test = np.concatenate((image_test, data[-testsize:]), axis=0) if image_test is not None else data[-testsize:]
    image_val = np.concatenate((image_val, data[-testsize-valsize:-testsize]), axis=0) if image_val is not None else data[-testsize-valsize:-testsize]
    image_train = np.concatenate((image_train, data[:-testsize-valsize]), axis=0) if image_train is not None else data[:-testsize-valsize]
    category_test.extend([i] * (testsize))
    category_val.extend([i] * (valsize))
    category_train.extend([i] * (instance_num-testsize-valsize))
    

In [ ]:
# save data
with open(labelpath.format('train'), 'w') as fd:
    json.dump(category_train, fd)
with open(labelpath.format('val'), 'w') as fd:
    json.dump(category_val, fd)
with open(labelpath.format('test'), 'w') as fd:
    json.dump(category_test, fd)
    
np.save(datapath.format('train'), image_train)
np.save(datapath.format('val'), image_val)
np.save(datapath.format('test'), image_test)

In [6]:
len(name_dict)

345